In [1]:
using Random
using Random123
using Distributions
using Plots
using StatsPlots
using StatsBase
using Statistics
using DataFrames
using Chain
using Dates
using CSV
include("Utils.jl")

initialize_prngs

## 08 Intermediate Epistasis Static Landscapes
We now move to the final case, where landscapes are semi-correlated.

In [3]:
total_population = 1000
σ_epi::Float64 = 0.1 # We've reduced this to lessen the selection coefficient
μ = (total_population^-1)/10 # Mutation rate of the genotypes with some genome. Claudia says than Nμ = 1 is a weird parameter regime, so we adjust it a bit lower
M = 0.0 # For the static versions
simulation_length = 50000
final_genome = DataFrame(Loci = Int[], Replicate = Int[], Final_fitness = Float64[], Final_genome_size = Float64[]) # unlike with the additive landscape, there is no trivial way of determining max fitness.
runtime = Dates.format(now(), "yyyymmdd_HHMM")
for i in 1:200
    print("\r$i")
    for test in (1,25,50,75,100)
        try
            loci = 100
            init_active_loci = test
            max_init_genotype_bits = 1
            rng_default, rng_additive, rng_init_genotype, rng_init_genome, rng_mutation = initialize_prngs(genome_seed = i, mutation_seed = i) # changing the genome seed changes the landscape, since the cbrng is dependent on the genome 
            additive_effects = generate_additive_effects(rng_additive, 0.1, 128) # we are also reducing the additive stdev to lower selection coeff
            df_genotypes = simulate(loci, init_active_loci, max_init_genotype_bits, total_population, σ_epi, μ, M, simulation_length, rng_init_genome, rng_init_genotype, rng_default, rng_mutation, additive_effects)
            #generate_plots(df_genotypes)
            sizes, sweeps = process_data(df_genotypes, μ, M, additive_effects, σ_epi)[6:7]
            if nrow(sweeps) > 0
                final_fitness = get_fitness(sweeps[!,:Genotype][end], sweeps[!,:Genome][end], additive_effects, σ_epi)
                final_genome_size = sizes[end, :AverageGenomeSize]
                push!(final_genome, (test, i, final_fitness, final_genome_size)) # genome size of major genotype at end of simulation
                CSV.write("outputs/data/08_intermediate_static_landscape_$(runtime).csv", final_genome)
            else 
                NaN
            end
            
        catch e
            println("\nError at replicate=$i, init_active_loci=$test")
            println(e)
            println(stacktrace(catch_backtrace()))
        end
    end
end

println("\nFinished at: ", Dates.format(now(), "yyyymmdd_HHMM"))

Iteration: 46000

Excessive output truncated after 524301 bytes.

## 09 Intermediate Epistasis Expanding Landscapes

In [ ]:
total_population = 1000
σ_epi::Float64 = 0.1 # We've reduced this to lessen the selection coefficient
μ = (total_population^-1)/10 # Mutation rate of the genotypes with some genome. Claudia says than Nμ = 1 is a weird parameter regime, so we adjust it a bit lower
M = μ * 10^-1
simulation_length = 50000
final_genome = DataFrame(Loci = Int[], Replicate = Int[], Final_fitness = Float64[], Final_genome_size = Float64[]) # unlike with the additive landscape, there is no trivial way of determining max fitness.
runtime = Dates.format(now(), "yyyymmdd_HHMM")
for i in 1001:2000
    print("\r$i")
    for test in (5,6,7,8,9,10)
        try
            loci = 20
            init_active_loci = test
            max_init_genotype_bits = 1
            rng_default, rng_additive, rng_init_genotype, rng_init_genome, rng_mutation = initialize_prngs(genome_seed = i, mutation_seed = i) # changing the genome seed changes the landscape, since the cbrng is dependent on the genome 
            additive_effects = generate_additive_effects(rng_additive, 0.1, 128) # we are also reducing the additive stdev to lower selection coeff
            df_genotypes = simulate(loci, init_active_loci, max_init_genotype_bits, total_population, σ_epi, μ, M, simulation_length, rng_init_genome, rng_init_genotype, rng_default, rng_mutation, additive_effects)
            #generate_plots(df_genotypes)
            sizes, sweeps = process_data(df_genotypes, μ, M, additive_effects, σ_epi)[6:7]
            if nrow(sweeps) > 0
                final_fitness = get_fitness(sweeps[!,:Genotype][end], sweeps[!,:Genome][end], additive_effects, σ_epi)
                final_genome_size = sizes[end, :AverageGenomeSize]
                push!(final_genome, (test, i, final_fitness, final_genome_size)) # genome size of major genotype at end of simulation
                CSV.write("outputs/data/09_intermediate_expanding_landscape_$(runtime).csv", final_genome)
            else 
                NaN
            end
            
        catch e
            println("\nError at replicate=$i, init_active_loci=$test")
            println(e)
            println(stacktrace(catch_backtrace()))
        end
    end
end

println("\nFinished at: ", Dates.format(now(), "yyyymmdd_HHMM"))

Iteration: 20000

Excessive output truncated after 524315 bytes.

## 10 Testing Dynamics on Expanding Landscapes

In [2]:
total_population = 1000
σ_epi::Float64 = 0.1 # We've reduced this to lessen the selection coefficient
μ = (total_population^-1)/10 # Mutation rate of the genotypes with some genome. Claudia says than Nμ = 1 is a weird parameter regime, so we adjust it a bit lower
M = μ * 10^-1
simulation_length = 50000
final_genome = DataFrame(Loci = Int[], Replicate = Int[], Final_fitness = Float64[], Final_genome_size = Float64[]) # unlike with the additive landscape, there is no trivial way of determining max fitness.
output_dataframe = DataFrame(Step = Int[], Fitness = Float64[], CurrentGenomeSize = Float64[], FinalGenomeSize = Float64[], StartingLoci = Int[], Replicate = Int[])
runtime = Dates.format(now(), "yyyymmdd_HHMM")
for i in 1:5000
    print("\r$i")
    for test in (5)
        try
            loci = 20
            init_active_loci = test
            max_init_genotype_bits = 1
            rng_default, rng_additive, rng_init_genotype, rng_init_genome, rng_mutation = initialize_prngs(genome_seed = i, mutation_seed = i) # changing the genome seed changes the landscape, since the cbrng is dependent on the genome 
            additive_effects = zeros(128)#generate_additive_effects(rng_additive, 0.1, 128) # we are also reducing the additive stdev to lower selection coeff
            df_genotypes = simulate(loci, init_active_loci, max_init_genotype_bits, total_population, σ_epi, μ, M, simulation_length, rng_init_genome, rng_init_genotype, rng_default, rng_mutation, additive_effects)
            #generate_plots(df_genotypes)
            sizes, sweeps = process_data(df_genotypes, μ, M, additive_effects, σ_epi)[6:7]
            if nrow(sweeps) > 0
                #final_fitness = get_fitness(sweeps[!,:Genotype][end], sweeps[!,:Genome][end], additive_effects, σ_epi)
                sweeps = sweeps[:, [:Step, :Fitness, :CurrentGenomeSize]]
                sweeps[!, :FinalGenomeSize] .= sizes[end, :AverageGenomeSize]
                sweeps[!, :StartingLoci] .= test
                sweeps[!, :Replicate] .= i
                #final_genome_size = sizes[end, :AverageGenomeSize]
                output_dataframe = vcat(output_dataframe, sweeps) # genome size of major genotype at end of simulation
                CSV.write("outputs/data/10-1_dynamics_hoc_expanding_landscape_$(runtime).csv", output_dataframe)
            else 
                NaN
            end
            
        catch e
            println("\nError at replicate=$i, init_active_loci=$test")
            println(e)
            println(stacktrace(catch_backtrace()))
        end
    end
end

println("\nFinished at: ", Dates.format(now(), "yyyymmdd_HHMM"))

Iteration: 29000

Excessive output truncated after 524314 bytes.

## 11 Testing Individual Cases

In [16]:
loci = 20
init_active_loci = 1
max_init_genotype_bits = 1
total_population = 1000
σ_epi::Float64 = 0.1
μ = (total_population^-1)/10
M = μ * 10^-1
simulation_length = 10000
i = 5
#for init_active_loci in (1,10,20)
rng_default, rng_additive, rng_init_genotype, rng_init_genome, rng_mutation = initialize_prngs(genome_seed = i, mutation_seed = i)
additive_effects = generate_additive_effects(rng_additive, 0.1, 128)
df_genotypes = simulate(loci, init_active_loci, max_init_genotype_bits, total_population, σ_epi, μ, M, simulation_length, rng_init_genome, rng_init_genotype, rng_default, rng_mutation, additive_effects)
#generate_plots(df_genotypes, μ, M, additive_effects, σ_epi, save = false)
output_data = process_data(df_genotypes, μ, M, additive_effects, σ_epi)[7]
#insertcols!(df, string(init_active_loci) => output_data[!,:AverageGenomeSize])
#final_fitness = get_fitness(output_data[!,:Genotype][end], output_data[!,:Genome][end], additive_effects, σ_epi)
#max_fitness = exp(sum(heaviside.(additive_effects[1:loci]) .* additive_effects[1:loci]))
#end
#CSV.write("output/additive_replicate124_data_20250724.csv", df)

Iteration: 10000

Row,Step,Genotype,Genome,Pop,GenotypeChange,GenomeChange,Fitness,GenomeSize
,Int64,UInt128,UInt128,Float64,Float64,Float64,Float64,Int64
1,0,0,524288,1000.0,0.0,0.0,1.02354,1
2,110,0,786432,515.0,0.0,1.0,1.18577,2
3,1112,0,802816,502.0,0.0,1.0,1.22188,3
4,3828,0,802820,519.0,0.0,1.0,1.25167,4
